In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

module_path = os.path.abspath(os.path.join('..', 'code'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import utils

In [3]:
data = utils.load_data('../data/output.csv')

FileNotFoundError: [Errno 2] No such file or directory: '../data/output.csv'

In [ ]:
data.sample(5)

,Evento,Puerta,FechaHora,Rol
67973,Acceso concedido,SD9-MOL3-IN-T9,2024/10/01 12:13:52,ASEO
27150,Acceso concedido,ML3-MOL1-IN-T2,2024/10/01 13:35:46,EMPLEADO
17201,Acceso concedido,ML-VEHICULAR-ML-T9,2024/10/01 18:38:18,EMPLEADO
41317,Acceso concedido,SD-VEHICULAR-SD-T2,2024/10/01 08:31:15,ESTUDIANTE
79079,Acceso concedido,LL8-MOL1-OUT-T7,2024/10/01 17:59:15,ESTUDIANTE


In [ ]:
data['custom_index'] = (data['Rol']).apply(utils.create_index)
data = data.sort_values(by='FechaHora').reset_index().drop(columns='index')
data.head(100000)

,Evento,Puerta,FechaHora,Rol,custom_index
0,Acceso concedido,RGD11-MOL3-IN-T2,2024/10/01 00:00:16,VIGILANCIA,19063
1,Acceso concedido,LL11-MOL4-IN-T1,2024/10/01 00:00:19,ASEO,16291
2,Acceso concedido,SD4-MOL2-IN-T10,2024/10/01 00:00:31,VIGILANCIA,19846
3,Acceso concedido,ML-VEHICULAR-ML-T7,2024/10/01 00:00:59,ASEO,16445
4,Acceso concedido,SD-VEHICULAR-SD-T5,2024/10/01 00:01:04,VIGILANCIA,19773
...,...,...,...,...,...
99995,Acceso concedido,LL12-MOL3-IN-T6,2024/10/01 23:59:51,ESTUDIANTE,5150
99996,Acceso concedido,CD5-MOL5-OUT-T9,2024/10/01 23:59:52,ESTUDIANTE,14008
99997,Acceso concedido,ML-VEHICULAR-ML-T11,2024/10/01 23:59:52,VIGILANCIA,19246
99998,Acceso concedido,SD4-MOL4-IN-T5,2024/10/01 23:59:53,ESTUDIANTE,13212


In [ ]:
import cv2
import numpy as np
import pytesseract

# Configurar la ruta de Tesseract-OCR (asegúrate de que esté instalado)
# pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'  # For Windows

# Cargar las imágenes
images = [
    cv2.imread('../assets/Piso1.png'),
    cv2.imread('../assets/Piso2.png'),
    cv2.imread('../assets/Piso5.png'),
    cv2.imread('../assets/Piso6.png'),
]

for i, image in enumerate(images):

    # Convertir la imagen a espacio de color HSV para facilitar la detección de colores
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Definir los límites del color rojo en el espacio HSV
    lower_red1 = np.array([0, 50, 50])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([170, 50, 50])
    upper_red2 = np.array([180, 255, 255])

    # Crear máscaras para ambos rangos de rojo
    mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
    mask2 = cv2.inRange(hsv, lower_red2, upper_red2)

    # Unir las dos máscaras para cubrir todas las tonalidades de rojo
    mask = mask1 + mask2

    # Encontrar contornos en la máscara
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Crear una lista para almacenar las coordenadas de los cuadrados
    square_coords = []

    # Filtrar solo los contornos que sean cuadrados y que tengan un área significativa
    for contour in contours:
        # Aproximar el contorno
        epsilon = 0.02 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)

        # Verificar si el contorno tiene 4 lados (es un cuadrado o rectángulo)
        if len(approx) == 4 and cv2.contourArea(contour) > 100:  # Filtrar por área
            square_coords.append(approx)

            # Extraer la región del cuadrado detectado de la imagen original
            x, y, w, h = cv2.boundingRect(approx)
            square_roi = image[y:y + h, x:x + w]

            # Preprocesar la imagen antes de OCR (convertir a gris y aplicar umbral)
            gray_square = cv2.cvtColor(square_roi, cv2.COLOR_BGR2GRAY)

            # Binarizar la imagen (solo si es necesario)
            _, threshold_square = cv2.threshold(gray_square, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

            # Usar OCR para leer el texto dentro del cuadrado (configurado para una línea)
            text = pytesseract.image_to_string(threshold_square, config='--psm 7')  # psm 7 para una línea de texto

            # Decidir el color basado en el texto
            if "WAIRA" in text.upper():
                color = (0, 255, 0)  # Verde si el texto es "APPROVED"
            elif "ML-603" in text.upper():
                color = (0, 0, 255)  # Rojo si el texto es "REJECTED"
            else:
                color = (255, 255, 0)  # Amarillo para otros casos

            # Dibujar el cuadrado completo con el color basado en el texto
            cv2.fillPoly(image, [approx], color)

            # Poner el texto extraído en la imagen
            cv2.putText(image, text.strip(), (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2, cv2.LINE_AA)

    # Guardar la imagen con los cuadrados detectados y coloreados
    cv2.imwrite(f'../assets/out/{i}.jpg', image)

    # Mostrar la imagen con los cuadrados y textos detectados
    cv2.imshow('Cuadrados con Texto Detectado', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


    # # Imprimir las coordenadas de las esquinas de los cuadrados detectados
    # for i, square in enumerate(square_coords):
    #     print(f'Cuadrado {i+1}:')
    #     for point in square:
    #         x, y = point[0]
    #         print(f'    Esquina: (x: {x}, y: {y})')


## Procesado de datos Oferta de cursos

In [ ]:
#%pip install polars
import polars as pl
from ast import literal_eval

In [ ]:
df = pl.read_csv('C:/Users/carlo/Desktop/2024-2/Tesis/Thesis-SmartCampusVisualization/scrap/formatted_data.csv', infer_schema_length=10000)

In [ ]:
df.sample(5)

Course Code,Course Name,Cupo,NRC,Sección,Créditos,Periodo,Parte de Periodo,Campus,Schedule Table,Cupos Ofrecidos,Cupos Inscritos,Cupos Disponibles
str,str,str,i64,str,i64,i64,str,str,str,i64,i64,i64
"""ARTE-2153""","""SERIGRAFÍA - ESTÉNCIL""","""2""",52958,"""01""",4,202420,"""PERIODO 202420 - 16 SEMANAS""","""CAMPUS PRINCIPAL""","""[{'Día': 'M', 'Hora': '1400-17…",15,13,2
"""QUIM-1103""","""QUÍMICA""","""6""",10700,"""B""",3,202420,"""PERIODO 202420 - 16 SEMANAS""","""CAMPUS PRINCIPAL""","""[{'Día': 'M', 'Hora': 'J', 'Sa…",64,58,6
"""QUIM-4413""","""BIOQUÍMICA AVANZADA (CICLO 1 D…","""6""",71874,"""01""",2,202420,"""PRIMER CICLO - 8 SEMANAS""","""CAMPUS PRINCIPAL""","""[{'Día': 'I', 'Hora': 'V', 'Sa…",15,9,6
"""IMEC-2006""","""TALLER DE MODELADO ESTRUCTURAL…","""2""",72784,"""01""",2,202420,"""PRIMER CICLO - 8 SEMANAS""","""CAMPUS PRINCIPAL""","""[{'Día': 'M', 'Hora': 'J', 'Sa…",19,17,2
"""INTL-2218""","""INTERCAMBIO NACIONAL 8""","""5""",70372,"""01""",1,202420,"""PERIODO NO RETIRABLE""","""EXTERNO UNIVERSIDAD""","""[{'Día': '-', 'Hora': '.NOREQ'…",5,0,5


In [ ]:
# change the column type to list
df = df.with_columns(
    pl.col("Schedule Table").map_elements(literal_eval).alias("Schedule Table")
)

sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredict

In [ ]:
df.sample(10)

Course Code,Course Name,NRC,Parte de Periodo,Campus,Schedule Table,Cupos Inscritos
str,str,i64,str,str,list[struct[6]],i64
"""BIOE-4102""","""DERECHO A LA SALUD (CICLO 2 DE…",70438,"""SEGUNDO CICLO - 8 SEMANAS""","""VIRTUAL""","[{""M"",""J"",""1730-2020"",""."",""2024-10-07"",""2024-11-30 .""}]",16
"""ICYA-4602""","""DISEÑO AVANZADO DE PAVIMENTOS""",71254,"""PERIODO 202420 - 16 SEMANAS""","""CAMPUS PRINCIPAL""","[{""L"",""I"",""1530-1650"","".RGD_202"",""2024-08-05"",""2024-11-30 .centro cívico""}]",20
"""LENG-1157""","""ENGLISH 07 - SPEAKING 1 (CICLO…",27931,"""PRIMER CICLO - 8 SEMANAS""","""CAMPUS PRINCIPAL""","[{""M"",""I"",""J"",""V"",""0630-0750"","".AU_402 2024-08-05 2024-09-28 .Edif. Aulas""}]",19
"""MATE-2303""","""ECUACIONES DIFERENCIALES (INGE…",30443,"""PERIODO 202420 - 16 SEMANAS""","""CAMPUS PRINCIPAL""","[{""M"",""J"",""1100-1220"","".RGD_212-13"",""2024-08-05"",""2024-11-30 .centro cívico""}]",58
"""CBCA-1333""","""VIVIR UNA VIDA ÉTICA (CICLO 1 …",71930,"""PRIMER CICLO - 8 SEMANAS""","""CAMPUS PRINCIPAL""","[{""M"",""J"",""1200-1350"","".O_301"",""2024-08-05"",""2024-09-28 .Edif. Henry Yerly (O)""}]",45
"""IBIO-4780""","""TESIS I""",72439,"""PERIODO 202420 - 16 SEMANAS""","""CAMPUS PRINCIPAL""","[{""-"",""."",""2024-08-05"",""2024-11-30"",""."",""No info""}]",1
"""HIST-4707""","""LECTURAS DIRIGIDAS 2""",71210,"""PERIODO 202420 - 16 SEMANAS""","""CAMPUS PRINCIPAL""","[{""-"",""."",""2024-08-05"",""2024-11-30"",""."",""No info""}]",1
"""FISI-1538""","""ONDAS Y FLUIDOS""",44900,"""PERIODO 202420 - 16 SEMANAS""","""CAMPUS PRINCIPAL""","[{""M"",""J"",""0800-0920"","".RGD_112-13"",""2024-08-05"",""2024-11-30 .centro cívico""}]",47
"""ARQT-2204""","""UNC PROYECTO DE LOS SISTEMAS D…",52219,"""PRIMER CICLO - 8 SEMANAS""","""CAMPUS PRINCIPAL""","[{""M"",""J"",""0700-0950"","".C_605"",""2024-08-05"",""2024-09-28 .Bloque C""}]",9


In [ ]:
#Drop the columns which are not needed
df = df.drop("Cupo", "Sección", "Créditos", "Periodo","Cupos Ofrecidos", "Cupos Disponibles")

In [ ]:
df_virt = pl.DataFrame()
df_virt = df.filter(pl.col('Campus').is_in(["NO DEFINIDO",
"VIRTUAL",
"EXTERNO UNIVERSIDAD" ] ))
df_virt.sample(10)

Course Code,Course Name,NRC,Parte de Periodo,Campus,Schedule Table,Cupos Inscritos
str,str,i64,str,str,list[struct[6]],i64
"""ARTI-4204""","""ARQUITECTURA DE SEGURIDAD (BC)""",73372,"""PERIODO 202420 - 16 SEMANAS""","""VIRTUAL""","[{""J"",""1500-1750"","".VIRT"",""2024-08-05"",""2024-11-30"","".VIRTUAL""}]",25
"""MEDI-3416C""","""GINECOLOGÍA Y OBSTETRICIA CONS…",73063,"""CURSOS MEDICINA - 21 SEMANAS""","""EXTERNO UNIVERSIDAD""","[{""L"",""I"",""V"",""0700-1150"","".FSFB"",""2024-10-07 2024-10-13 .FUNDACION SANTAFE DE BOGOTA""}, {""M"",""J"",""0700-1550"","".FSFB"",""2024-10-07"",""2024-10-13 .FUNDACION SANTAFE DE BOGOTA""}]",3
"""MEDI-3416F""","""GINECOLOGÍA Y OBSTETRICIA PROF…",73078,"""CURSOS MEDICINA - 21 SEMANAS""","""EXTERNO UNIVERSIDAD""","[{""M"",""J"",""0700-1150"","".PROFAM"",""2024-11-05"",""2024-11-10 .PROFAMILIA""}, {""I"",""V"",""0700-1150"","".PROFAM"",""2024-11-05"",""2024-11-10 .PROFAMILIA""}]",2
"""CISO-4032""","""HISTORIA Y SOCIEDAD GLOBAL""",70778,"""PERIODO 202420 - 16 SEMANAS""","""VIRTUAL""","[{""J"",""1600-1850"","".VIRT"",""2024-08-05"",""2024-11-30"","".VIRTUAL""}]",23
"""MEDI-3413C""","""ORTOPEDÍA CONSULTA EXTERNA""",69802,"""CURSOS MEDICINA - 21 SEMANAS""","""EXTERNO UNIVERSIDAD""","[{""L"",""I"",""0700-1550"","".FSFB"",""2024-09-23"",""2024-09-29 .FUNDACION SANTAFE DE BOGOTA""}, {""M"",""J"",""0700-1250"","".FSFB"",""2024-09-23"",""2024-09-29 .FUNDACION SANTAFE DE BOGOTA""}, {""V"",""D"",""0700-1220"","".FSFB"",""2024-09-23"",""2024-09-29 .FUNDACION SANTAFE DE BOGOTA""}]",4
"""MEDI-3407U""","""PEDIATRÍA URGENCIAS""",69648,"""CURSOS MEDICINA - 21 SEMANAS""","""EXTERNO UNIVERSIDAD""","[{""L"",""I"",""J"",""0700-1550"","".HTINTAL"",""2024-09-23 2024-10-13 .HOSPITAL EL TINTAL""}, {""M"",""V"",""0700-1150"","".HTINTAL"",""2024-09-23"",""2024-10-13 .HOSPITAL EL TINTAL""}, {""D"",""0700-1850"","".HTINTAL"",""2024-09-23"",""2024-10-13"","".HOSPITAL EL TINTAL""}]",4
"""MEDI-3417R""","""MEDICINA LEGAL ROTACIÓN""",70041,"""CURSOS MEDICINA - 21 SEMANAS""","""EXTERNO UNIVERSIDAD""","[{""L"",""I"",""V"",""0700-1150"","".MEDLEG"",""2024-10-28 2024-11-10 .MEDICINA LEGAL""}, {""M"",""J"",""0700-1550"","".MEDLEG"",""2024-10-28"",""2024-11-10 .MEDICINA LEGAL""}, {""D"",""0700-1850"","".MEDLEG"",""2024-10-28"",""2024-11-10"","".MEDICINA LEGAL""}]",6
"""FILO-3010""","""CODIGO DE MONOGRAFIA""",10498,"""PERIODO NO RETIRABLE""","""NO DEFINIDO""","[{""-"","".NOREQ"",""2024-08-05"",""2024-11-30"","".No"",""Requiere""}]",4
"""CBPC-1026""","""ARTE Y CIENCIAS""",63734,"""PERIODO 202420 - 16 SEMANAS""","""VIRTUAL""","[{""M"",""0900-1050"","".VIRT"",""2024-08-05"",""2024-11-30"","".VIRTUAL""}]",55
